In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
# Load the data
# df = pd.read_csv('Exploratory/untitled - 2025-11-18T141129.591.csv')



# Load the machine-day-sensortype level data 

df_motorbelasting=pd.read_csv('/Users/roelpost/DeveloperTools/MWB bubble chart/Exploratory/Motorbelasting.csv')


df_motorbelasting.head()

,MachineId,source_origin_id,FK_MDMsensor_Sys_id,datekey,countRows,totalHoursPerDay,totalFuelPerDay,totalHoursAbove20p,FPH,maxFPH,...,SubGroupLabel,BrandLabel,TypeOfEquipment,displayname_unique,Machinegroep,EngineClassificationLabel,ConstructionYear,Power,SensorSupplier,SensorType
0,710,291941,32867a3b97b1902423c86a5c91c07742950a101a027517...,20250323,471,7.980556,36.844461,4.692580,4.616777,18.6,...,Tandemtrilrolwals,Hamm,HX90i VS-OS,Hamm HX90i VS-OS 710,D,Stage-V,2023,74.4,GPS Buddy,Fuel_consumption
1,710,291941,32867a3b97b1902423c86a5c91c07742950a101a027517...,20250327,384,6.618889,27.146380,4.085070,4.101351,18.6,...,Tandemtrilrolwals,Hamm,HX90i VS-OS,Hamm HX90i VS-OS 710,D,Stage-V,2023,74.4,GPS Buddy,Fuel_consumption
2,710,291941,32867a3b97b1902423c86a5c91c07742950a101a027517...,20250404,249,4.589722,16.013058,2.511426,3.488894,18.6,...,Tandemtrilrolwals,Hamm,HX90i VS-OS,Hamm HX90i VS-OS 710,D,Stage-V,2023,74.4,GPS Buddy,Fuel_consumption
3,710,291941,32867a3b97b1902423c86a5c91c07742950a101a027517...,20250422,148,2.500000,13.183326,1.816699,5.273333,18.6,...,Tandemtrilrolwals,Hamm,HX90i VS-OS,Hamm HX90i VS-OS 710,D,Stage-V,2023,74.4,GPS Buddy,Fuel_consumption
4,710,291941,32867a3b97b1902423c86a5c91c07742950a101a027517...,20250417,54,0.916667,4.716666,0.650009,5.145455,18.6,...,Tandemtrilrolwals,Hamm,HX90i VS-OS,Hamm HX90i VS-OS 710,D,Stage-V,2023,74.4,GPS Buddy,Fuel_consumption


In [ ]:
# Create machine-day level consolidated dataframe
print("=" * 60)
print("CREATING MACHINE-DAY CONSOLIDATED DATAFRAME")
print("=" * 60)

# First, let's check what sensor types we have
print("\nAvailable sensor types:")
print(df_motorbelasting['SensorType'].value_counts())

# Pivot the data to have one column per sensor type
df_pivoted = df_motorbelasting.pivot_table(
    index=['MachineId', 'datekey', 'source_origin_id', 'MainGroupLabel', 'SubGroupLabel', 
           'BrandLabel', 'TypeOfEquipment', 'displayname_unique', 'Machinegroep',
           'EngineClassificationLabel', 'ConstructionYear', 'Power'],
    columns='SensorType',
    values=['totalFuelPerDay', 'totalHoursPerDay', 'FPH', 'motorbelasting', 'countRows'],
    aggfunc='first'  # Use first value if there are duplicates
).reset_index()

# Flatten column names
df_pivoted.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in df_pivoted.columns.values]

# Create consolidated fuel column with priority: Fuel_consumption > Fuel flow > CANBUS
df_pivoted['consolidated_fuel'] = np.nan
df_pivoted['fuel_source'] = None

# Check which fuel columns exist and apply hierarchy
fuel_consumption_col = 'totalFuelPerDay_Fuel_consumption'
fuel_flow_col = 'totalFuelPerDay_Fuel flow'
canbus_col = 'totalFuelPerDay_CANBUS'

# Priority 1: Fuel_consumption
if fuel_consumption_col in df_pivoted.columns:
    mask = df_pivoted[fuel_consumption_col].notna()
    df_pivoted.loc[mask, 'consolidated_fuel'] = df_pivoted.loc[mask, fuel_consumption_col]
    df_pivoted.loc[mask, 'fuel_source'] = 'Fuel_consumption'
    print(f"\n✓ Applied Fuel_consumption for {mask.sum():,} rows")

# Priority 2: Fuel flow (only if Fuel_consumption is not available)
if fuel_flow_col in df_pivoted.columns:
    mask = (df_pivoted['consolidated_fuel'].isna()) & (df_pivoted[fuel_flow_col].notna())
    df_pivoted.loc[mask, 'consolidated_fuel'] = df_pivoted.loc[mask, fuel_flow_col]
    df_pivoted.loc[mask, 'fuel_source'] = 'Fuel flow'
    print(f"✓ Applied Fuel flow for {mask.sum():,} rows")

# Priority 3: CANBUS (only if both above are not available)
if canbus_col in df_pivoted.columns:
    mask = (df_pivoted['consolidated_fuel'].isna()) & (df_pivoted[canbus_col].notna())
    df_pivoted.loc[mask, 'consolidated_fuel'] = df_pivoted.loc[mask, canbus_col]
    df_pivoted.loc[mask, 'fuel_source'] = 'CANBUS'
    print(f"✓ Applied CANBUS for {mask.sum():,} rows")

# Similarly consolidate hours and motorbelasting
# Hours - use same priority
df_pivoted['consolidated_hours'] = np.nan
for sensor_type in ['Fuel_consumption', 'Fuel flow', 'CANBUS']:
    hours_col = f'totalHoursPerDay_{sensor_type}'
    if hours_col in df_pivoted.columns:
        mask = df_pivoted['consolidated_hours'].isna() & df_pivoted[hours_col].notna()
        df_pivoted.loc[mask, 'consolidated_hours'] = df_pivoted.loc[mask, hours_col]

# Motorbelasting - use same priority
df_pivoted['consolidated_motorbelasting'] = np.nan
for sensor_type in ['Fuel_consumption', 'Fuel flow', 'CANBUS']:
    mb_col = f'motorbelasting_{sensor_type}'
    if mb_col in df_pivoted.columns:
        mask = df_pivoted['consolidated_motorbelasting'].isna() & df_pivoted[mb_col].notna()
        df_pivoted.loc[mask, 'consolidated_motorbelasting'] = df_pivoted.loc[mask, mb_col]

# FPH - use same priority
df_pivoted['consolidated_FPH'] = np.nan
for sensor_type in ['Fuel_consumption', 'Fuel flow', 'CANBUS']:
    fph_col = f'FPH_{sensor_type}'
    if fph_col in df_pivoted.columns:
        mask = df_pivoted['consolidated_FPH'].isna() & df_pivoted[fph_col].notna()
        df_pivoted.loc[mask, 'consolidated_FPH'] = df_pivoted.loc[mask, fph_col]

print(f"\n{'='*60}")
print("CONSOLIDATION SUMMARY")
print(f"{'='*60}")
print(f"Total machine-day records: {len(df_pivoted):,}")
print(f"\nFuel source breakdown:")
print(df_pivoted['fuel_source'].value_counts())
print(f"\nRows with fuel data: {df_pivoted['consolidated_fuel'].notna().sum():,}")
print(f"Rows without fuel data: {df_pivoted['consolidated_fuel'].isna().sum():,}")

# Create final clean dataframe with selected columns
df_machine_day = df_pivoted[[
    'MachineId', 'datekey', 'source_origin_id', 'displayname_unique',
    'MainGroupLabel', 'SubGroupLabel', 'BrandLabel', 'TypeOfEquipment',
    'Machinegroep', 'EngineClassificationLabel', 'ConstructionYear', 'Power',
    'consolidated_fuel', 'consolidated_hours', 'consolidated_motorbelasting', 
    'consolidated_FPH', 'fuel_source'
]].copy()

# Rename consolidated columns for clarity
df_machine_day.rename(columns={
    'consolidated_fuel': 'fuel',
    'consolidated_hours': 'hours',
    'consolidated_motorbelasting': 'motorbelasting',
    'consolidated_FPH': 'FPH'
}, inplace=True)

print(f"\n{'='*60}")
print(f"Final dataframe shape: {df_machine_day.shape}")
print(f"{'='*60}")

df_machine_day.head(10)

In [ ]:
# Analyze fuel-related sensors
print("=" * 60)
print("FUEL SENSOR ANALYSIS")
print("=" * 60)

# Get all unique sensor types
print("\nAll sensor types in the dataset:")
print(df_motorbelasting['SensorType'].unique())

# Filter for fuel-related sensors
fuel_sensors = df_motorbelasting[df_motorbelasting['SensorType'].str.contains('fuel', case=False, na=False)]

print(f"\n\nFuel-related sensor types and row counts:")
print("-" * 60)
fuel_sensor_counts = fuel_sensors['SensorType'].value_counts()
print(fuel_sensor_counts)

print(f"\n\nTotal rows with fuel sensors: {len(fuel_sensors):,}")
print(f"Total rows in dataset: {len(df_motorbelasting):,}")
print(f"Percentage of fuel sensor rows: {len(fuel_sensors)/len(df_motorbelasting)*100:.1f}%")

# Show summary with additional details
print("\n\nDetailed breakdown by sensor type:")
print("-" * 60)
fuel_summary = fuel_sensors.groupby('SensorType').agg({
    'MachineId': 'nunique',
    'SensorSupplier': lambda x: x.unique().tolist(),
    'totalFuelPerDay': ['mean', 'sum', 'count']
}).round(2)

for sensor_type in fuel_sensor_counts.index:
    sensor_data = fuel_sensors[fuel_sensors['SensorType'] == sensor_type]
    print(f"\n{sensor_type}:")
    print(f"  • Total rows: {len(sensor_data):,}")
    print(f"  • Unique machines: {sensor_data['MachineId'].nunique()}")
    print(f"  • Sensor suppliers: {sensor_data['SensorSupplier'].unique().tolist()}")
    print(f"  • Avg fuel per day: {sensor_data['totalFuelPerDay'].mean():.2f}")
    print(f"  • Total fuel: {sensor_data['totalFuelPerDay'].sum():.2f}")

In [ ]:
# Create machine-day level consolidated dataframe
print("=" * 60)
print("CREATING MACHINE-DAY CONSOLIDATED DATAFRAME")
print("=" * 60)

# First, let's check what sensor types we have
print("\nAvailable sensor types:")
print(df_motorbelasting['SensorType'].value_counts())

# Pivot the data to have one column per sensor type
df_pivoted = df_motorbelasting.pivot_table(
    index=['MachineId', 'datekey', 'source_origin_id', 'MainGroupLabel', 'SubGroupLabel', 
           'BrandLabel', 'TypeOfEquipment', 'displayname_unique', 'Machinegroep',
           'EngineClassificationLabel', 'ConstructionYear', 'Power'],
    columns='SensorType',
    values=['totalFuelPerDay', 'totalHoursPerDay', 'FPH', 'motorbelasting', 'countRows'],
    aggfunc='first'  # Use first value if there are duplicates
).reset_index()

# Flatten column names
df_pivoted.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in df_pivoted.columns.values]

# Create consolidated fuel column with priority: Fuel_consumption > Fuel flow > CANBUS
df_pivoted['consolidated_fuel'] = np.nan
df_pivoted['fuel_source'] = None

# Check which fuel columns exist and apply hierarchy
fuel_consumption_col = 'totalFuelPerDay_Fuel_consumption'
fuel_flow_col = 'totalFuelPerDay_Fuel flow'
canbus_col = 'totalFuelPerDay_CANBUS'

# Priority 1: Fuel_consumption
if fuel_consumption_col in df_pivoted.columns:
    mask = df_pivoted[fuel_consumption_col].notna()
    df_pivoted.loc[mask, 'consolidated_fuel'] = df_pivoted.loc[mask, fuel_consumption_col]
    df_pivoted.loc[mask, 'fuel_source'] = 'Fuel_consumption'
    print(f"\n✓ Applied Fuel_consumption for {mask.sum():,} rows")

# Priority 2: Fuel flow (only if Fuel_consumption is not available)
if fuel_flow_col in df_pivoted.columns:
    mask = (df_pivoted['consolidated_fuel'].isna()) & (df_pivoted[fuel_flow_col].notna())
    df_pivoted.loc[mask, 'consolidated_fuel'] = df_pivoted.loc[mask, fuel_flow_col]
    df_pivoted.loc[mask, 'fuel_source'] = 'Fuel flow'
    print(f"✓ Applied Fuel flow for {mask.sum():,} rows")

# Priority 3: CANBUS (only if both above are not available)
if canbus_col in df_pivoted.columns:
    mask = (df_pivoted['consolidated_fuel'].isna()) & (df_pivoted[canbus_col].notna())
    df_pivoted.loc[mask, 'consolidated_fuel'] = df_pivoted.loc[mask, canbus_col]
    df_pivoted.loc[mask, 'fuel_source'] = 'CANBUS'
    print(f"✓ Applied CANBUS for {mask.sum():,} rows")

# Similarly consolidate hours and motorbelasting
# Hours - use same priority
df_pivoted['consolidated_hours'] = np.nan
for sensor_type in ['Fuel_consumption', 'Fuel flow', 'CANBUS']:
    hours_col = f'totalHoursPerDay_{sensor_type}'
    if hours_col in df_pivoted.columns:
        mask = df_pivoted['consolidated_hours'].isna() & df_pivoted[hours_col].notna()
        df_pivoted.loc[mask, 'consolidated_hours'] = df_pivoted.loc[mask, hours_col]

# Motorbelasting - use same priority
df_pivoted['consolidated_motorbelasting'] = np.nan
for sensor_type in ['Fuel_consumption', 'Fuel flow', 'CANBUS']:
    mb_col = f'motorbelasting_{sensor_type}'
    if mb_col in df_pivoted.columns:
        mask = df_pivoted['consolidated_motorbelasting'].isna() & df_pivoted[mb_col].notna()
        df_pivoted.loc[mask, 'consolidated_motorbelasting'] = df_pivoted.loc[mask, mb_col]

# FPH - use same priority
df_pivoted['consolidated_FPH'] = np.nan
for sensor_type in ['Fuel_consumption', 'Fuel flow', 'CANBUS']:
    fph_col = f'FPH_{sensor_type}'
    if fph_col in df_pivoted.columns:
        mask = df_pivoted['consolidated_FPH'].isna() & df_pivoted[fph_col].notna()
        df_pivoted.loc[mask, 'consolidated_FPH'] = df_pivoted.loc[mask, fph_col]

print(f"\n{'='*60}")
print("CONSOLIDATION SUMMARY")
print(f"{'='*60}")
print(f"Total machine-day records: {len(df_pivoted):,}")
print(f"\nFuel source breakdown:")
print(df_pivoted['fuel_source'].value_counts())
print(f"\nRows with fuel data: {df_pivoted['consolidated_fuel'].notna().sum():,}")
print(f"Rows without fuel data: {df_pivoted['consolidated_fuel'].isna().sum():,}")

# Create final clean dataframe with selected columns
df_machine_day = df_pivoted[[
    'MachineId', 'datekey', 'source_origin_id', 'displayname_unique',
    'MainGroupLabel', 'SubGroupLabel', 'BrandLabel', 'TypeOfEquipment',
    'Machinegroep', 'EngineClassificationLabel', 'ConstructionYear', 'Power',
    'consolidated_fuel', 'consolidated_hours', 'consolidated_motorbelasting', 
    'consolidated_FPH', 'fuel_source'
]].copy()

# Rename consolidated columns for clarity
df_machine_day.rename(columns={
    'consolidated_fuel': 'fuel',
    'consolidated_hours': 'hours',
    'consolidated_motorbelasting': 'motorbelasting',
    'consolidated_FPH': 'FPH'
}, inplace=True)

print(f"\n{'='*60}")
print(f"Final dataframe shape: {df_machine_day.shape}")
print(f"{'='*60}")

df_machine_day.head(10)